# Modeling and Simulation in Python

Chapter 2: Simulation

Copyright 2017 Allen Downey

License: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0)


We'll start with the same code we saw last time: the magic command that tells Jupyter where to put the figures, and the import statement that gets the functions defined in the `modsim` module.

In [1]:
# If you want the figures to appear in the notebook, 
# and you want to interact with them, use
# %matplotlib notebook

# If you want the figures to appear in the notebook, 
# and you don't want to interact with them, use
# %matplotlib inline

# If you want the figures to appear in separate windows, use
# %matplotlib qt5

%matplotlib qt5

from modsim import *

## More than one System object

Here's the code from the previous chapter, with two changes:

1. I've added DocStrings that explain what each function does, and what parameters it takes.

2. I've added a parameter named `system` to the functions so they work with whatever `System` object we give them, instead of always using `bikeshare`.  That will be useful soon when we have more than one `System` object.

In [2]:
def run_steps(system, num_steps=1, p1=0.5, p2=0.5):
    """Simulate the given number of time steps.
    
    system: bikeshare System object
    num_steps: number of time steps
    p1: probability of an Olin->Wellesley customer arrival
    p2: probability of a Wellesley->Olin customer arrival
    """
    for i in range(num_steps):
        step(system, p1, p2)
        plot_system(system)
        
def step(system, p1=0.5, p2=0.5):
    """Simulate one minute of time.
    
    system: bikeshare System object
    p1: probability of an Olin->Wellesley customer arrival
    p2: probability of a Wellesley->Olin customer arrival
    """
    if flip(p1):
        bike_to_wellesley(system)
    
    if flip(p2):
        bike_to_olin(system)
        
def bike_to_wellesley(system):
    """Move one bike from Olin to Wellesley.
    
    system: bikeshare System object
    """
    move_bike(system, 1)
    
def bike_to_olin(system):
    """Move one bike from Wellesley to Olin.
    
    system: bikeshare System object
    """
    move_bike(system, -1)
    
def move_bike(system, n):
    """Move a bike.
    
    system: bikeshare System object
    n: +1 to move from Olin to Wellesley or
       -1 to move from Wellesley to Olin
    """
    system.olin -= n
    system.wellesley += n
    
def plot_system(system):
    """Plot the current system of the bikeshare system.
    
    system: bikeshare System object
    """
    plot(system.olin, 'ro-', label='Olin')
    plot(system.wellesley, 'bo-', label='Wellesley')
    plot(system.olin_empty, 'gs-', label = 'O. Empty')
    plot(system.wellesley_empty, 's-', label = 'W. Empty')
    
def decorate():
    """Add a legend and label the axes.
    """
    legend(loc='best')
    label_axes(title='Olin-Wellesley Bikeshare',
               xlabel='Time step (min)', 
               ylabel='Number of bikes')

Now we can create more than one `System` object:

In [ ]:
bikeshare1 = System(olin=10, wellesley=2)
bikeshare1

In [4]:
bikeshare2 = System(olin=10, wellesley=2)
bikeshare2

,value
olin,10
wellesley,2


And whenever we call a function, we indicate which `System` object to work with:

In [5]:
bike_to_olin(bikeshare1)
bikeshare1

,value
olin,11
wellesley,1


In [6]:
bike_to_wellesley(bikeshare2)
bikeshare2

,value
olin,9
wellesley,3


And you can confirm that the different systems are getting updated independently:

In [7]:
bikeshare1

,value
olin,11
wellesley,1


In [8]:
bikeshare2

,value
olin,9
wellesley,3


## Negative bikes

In the code we have so far, the number of bikes at one of the locations can go negative, and the number of bikes at the other location can exceed the actual number of bikes in the system.

If you run this simulation a few times, it happens quite often.

In [14]:
bikeshare = System(olin=10, wellesley=2)
newfig()
plot_system(bikeshare)
decorate()
run_steps(bikeshare, 60, 0.8, 0.2)

But this is relatively easy to fix, using the `return` statement to exit the function early if the update would cause negative bikes.

If the second `if` statement seems confusing, remember that `n` can be negative.

In [3]:
def move_bike(system, n):
    # make sure the number of bikes won't go negative
    olin_temp = system.olin - n
    if olin_temp < 0:
        return
    
    wellesley_temp = system.wellesley + n
    if wellesley_temp < 0:
        return
    
    # update the system
    system.olin = olin_temp
    system.wellesley = wellesley_temp

Now if you run the simulation again, it should behave.

In [36]:
bikeshare = System(olin=10, wellesley=2)
newfig()
plot_system(bikeshare)
decorate()
run_steps(bikeshare, 60, 0.8, 0.2)

The variables `olin` and `wellesley` are created inside `move_bike`, so they are local.  When the function ends, they go away.

If you try to access a local variable from outside its function, you get an error:

In [37]:
# If you remove the # from the last line in this cell and run it, you'll get
# NameError: name 'olin' is not defined

olin

NameError: name 'olin' is not defined

**Exercise:** Add print statements in `move_bike` so it prints a message each time a customer arrives and doesn't find a bike.  Run the simulation again to confirm that it works as you expect.  Then you might want to remove the print statements before you go on.

In [4]:
def move_bike(system, n):
    olin_temp = system.olin - n
    if olin_temp < 0:
        print("There weren't enough bikes at Olin!")
        return
    wellesley_temp = system.wellesley + n
    if wellesley_temp < 0:
        print("There weren't enough bikes at Wellesley!")
        return
    system.olin = olin_temp
    system.wellesley = wellesley_temp

In [14]:
bikeshare = System(olin = 20, wellesley = 80)
newfig()
plot_system(bikeshare)
decorate()
run_steps(bikeshare, 60, .10, .50)


## Comparison operators

The `if` statements in the previous section used the comparison operator `<`.  The other comparison operators are listed in the book.

It is easy to confuse the comparison operator `==` with the assignment operator `=`.

Remember that `=` creates a variable or gives an existing variable a new value.

In [15]:
x = 5

Whereas `==` compared two values and returns `True` if they are equal.

In [16]:
x == 5

True

You can use `==` in an `if` statement.

In [17]:
if x == 5:
    print('yes, x is 5')

yes, x is 5


But if you use `=` in an `if` statement, you get an error.

In [18]:
# If you remove the # from the if statement and run it, you'll get
# SyntaxError: invalid syntax

if x >= 4:
    print('x is at least 4')
else:
    print('x is less than 4')

x is at least 4


**Exercise:** Add an `else` clause to the `if` statement above, and print an appropriate message.

Replace the `==` operator with one or two of the other comparison operators, and confirm they do what you expect.

## Metrics

Now that we have a working simulation, we'll use it to evaluate alternative designs and see how good or bad they are.  The metric we'll use is the number of customers who arrive and find no bikes available, which might indicate a design problem.

First we'll make a new `System` object that creates and initializes the system variables that will keep track of the metrics.

In [53]:
bikeshare = System(olin=10, wellesley=2, 
                  olin_empty=0, wellesley_empty=0)

Next we need a version of `move_bike` that updates the metrics.

In [5]:
def move_bike(system, n):
    olin_temp = system.olin - n
    if olin_temp < 0:
        system.olin_empty += 1
        return
    
    wellesley_temp = system.wellesley + n
    if wellesley_temp < 0:
        system.wellesley_empty += 1
        return
    
    system.olin = olin_temp
    system.wellesley = wellesley_temp

Now when we run a simulation, it keeps track of unhappy customers.

In [55]:
newfig()
plot_system(bikeshare)
decorate()
run_steps(bikeshare, 60, 0.4, 0.2)


After the simulation, we can print the number of unhappy customers at each location.

In [56]:
bikeshare.olin_empty

0

In [57]:
bikeshare.wellesley_empty

0

**Exercise:** Let's add a "clock" to keep track of how many time steps have elapsed:

1. Add a new system variable named `clock` to `bikeshare`, initialized to 0, and 

2. Modify `step` so it increments (adds one to) `clock` each time it is invoked.

Test your code by adding a print statement that prints the value of `clock` at the beginning of each time step.

In [6]:
# Here's a copy of step to get you started

bikeshare = System(olin = 10, wellesley = 2, olin_empty = 0, wellesley_empty = 0, clock = 0)

def step(system, p1=0.5, p2=0.5):
    """Simulate one minute of time.
    
    system: bikeshare System object
    p1: probability of an Olin->Wellesley customer arrival
    p2: probability of a Wellesley->Olin customer arrival
    """
    print (system.clock)
    if flip(p1):
        bike_to_wellesley(system)
    
    if flip(p2):
        bike_to_olin(system)
    
    system.clock += 1

In [59]:
run_steps(bikeshare, 60, 0.4, 0.2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [ ]:
# Solution goes here

In [ ]:
# Solution goes here

After the simulation, check the final value of `clock`.

In [60]:
print(bikeshare.clock)

60


**Exercise:** Now suppose we'd like to know how long it takes to run out of bikes at either location.  Modify `move_bike` so the first time a student arrives at Olin and doesn't find a bike, it records the value of `clock` in a system variable.

Hint: create a system variable named `t_first_empty` and initialize it to `None`, which is a special value (like `True` and `False`) that can be used to indicate a "special case".

Test your code by running a simulation for 60 minutes and checking the metrics.

In [7]:
def move_bike(system, n):
    olin_temp = system.olin - n
    if olin_temp < 0:
        if system.t_first_empty <0:
            system.t_first_empty = system.clock
        system.olin_empty += 1
        return
    
    wellesley_temp = system.wellesley + n
    if wellesley_temp < 0:
        system.wellesley_empty += 1
        return
    
    system.olin = olin_temp
    system.wellesley = wellesley_temp

def initialize_bikeshare():
    bikeshare = System(olin=10, wellesley=2, olin_empty=0, wellesley_empty=0, clock=0, t_first_empty=-1)    
    return bikeshare

In [64]:
bikeshare = initialize_bikeshare()
newfig()
plot_system(bikeshare)
decorate()
run_steps(bikeshare, 60, 0.4, 0.2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [65]:
print(bikeshare)

olin                0
wellesley          12
olin_empty          9
wellesley_empty     0
clock              60
t_first_empty      38
dtype: int64


In [70]:
if bikeshare.t_first_empty == -1:
    print('Olin never ran out of bikes.')
else:
    print('Olin ran out of bikes after', bikeshare.t_first_empty, 'minutes.')

Olin ran out of bikes after 38 minutes.


After the simulation, check the final value of `t_first_empty`.

In [31]:
# print(bikeshare.t_first_empty) (See above)

Before we go on, let's put `step` and `move_bike` back the way we found them, so they don't break the examples below. WOW WHAT A GREAT IDEA

In [8]:
def step(system, p1=0.5, p2=0.5):
    if flip(p1):
        bike_to_wellesley(system)
    
    if flip(p2):
        bike_to_olin(system)

def move_bike(system, n):
    olin_temp = system.olin - n
    if olin_temp < 0:
        system.olin_empty += 1
        return
    
    wellesley_temp = system.wellesley + n
    if wellesley_temp < 0:
        system.wellesley_empty += 1
        return
    
    system.olin = olin_temp
    system.wellesley = wellesley_temp

## Returning values

Here's a simple function that returns a value:

In [9]:
def add_five(x):
    return x + 5

And here's how we call it.

In [73]:
y = add_five(3)
y

8

If you run a function on the last line of a cell, Jupyter displays the result:

In [74]:
add_five(5)


10

But that can be a bad habit, because usually if you call a function and don't assign the result in a variable, the result gets discarded.

In the following example, Jupyter shows the second result, but the first result just disappears.

In [75]:
add_five(3)
add_five(5)

10

When you call a function that returns a variable, it is generally a good idea to assign the result to a variable.

In [76]:
y1 = add_five(3)
y2 = add_five(5)

print(y1, y2)

8 10


**Exercise:** Write a function called `init_system` that creates a `System` object with the system variables `olin=10` and `wellesley=2`, and then returns the new `System` object.

Write a line of code that calls `init_system` and assigns the result to a variable.

In [11]:
#WOW HOW DO I DO THAT
#Sorry, if anyone's reading this, I was kind of fighting with 
#initializing systems earlier because I hadn't learned how to do it right.
def init_system():
    system = System(olin = 10, wellesley = 2)
    return system     #see, that's what I was missing earlier.
    

In [85]:
init_system()   #so same deal here. If you run it 
                # at the end of your code, it prints.
print('a')

a


## Running simulations

Before we go on, I want to update `run_steps` so it doesn't always plot the results.  The new version takes an additional parameter, `plot_flag`, to indicate whether we want to plot.

"flag" is a conventional name for a boolean variable that indicates whether or not a condition is true.

This version of `run_steps` works even if `num_steps` is not an integer.  It uses the `int` function to round down.  See https://docs.python.org/3/library/functions.html#int

In [12]:
def run_steps(system, num_steps=1, p1=0.5, p2=0.5, plot_flag=True):
    """Simulate the given number of time steps.
    
    `num_steps` should be an integer; if not, it gets rounded down.
    
    system: bikeshare System object
    num_steps: number of time steps
    p1: probability of an Olin->Wellesley customer arrival
    p2: probability of a Wellesley->Olin customer arrival
    plot_flag: boolean, whether to plot
    """
    for i in range(int(num_steps)):
        step(system, p1, p2)
        if plot_flag:
            plot_system(system)

Now when we run a simulation, we can choose not to plot the results:

In [88]:
bikeshare = System(olin=10, wellesley=2, 
                  olin_empty=0, wellesley_empty=0)
run_steps(bikeshare, 60, 0.4, 0.2, plot_flag=False)

But after the simulation, we can still read the metrics.

In [89]:
bikeshare.olin_empty

0

Let's wrap all that in a function.

In [13]:
def run_simulation():
    system = System(olin=10, wellesley=2, 
                  olin_empty=0, wellesley_empty=0)
    run_steps(system, 60, 0.4, 0.2, plot_flag=False)
    return system

And test it.

In [93]:
system = run_simulation()

In [94]:
print(system.olin_empty, system.wellesley_empty)

11 0


If we generalize `run_simulation` to take `p1` and `p2`, we can use it to run simulations with a range of values for the parameters.

In [14]:
def run_simulation(p1=0.4, p2=0.2):
    bikeshare = System(olin=10, wellesley=2, 
                  olin_empty=0, wellesley_empty=0)
    run_steps(bikeshare, 60, p1, p2, plot_flag=False)
    return bikeshare

When `p1` is small, we probably don't run out of bikes at Olin.

In [171]:
system = run_simulation(p1=0.2)
system.olin_empty

0

When `p1` is large, we probably do.

In [172]:
system = run_simulation(p1=0.6)
system.olin_empty

8

**Exercise:**  Write a version of `run_simulation` that takes all five model parameters as function parameters.

In [15]:
def run_simulation(system=bikeshare, num_steps=60, p1=0.4, p2=0.2, plot_flag=False):
    system = System(olin=10, wellesley=2, 
                  olin_empty=0, wellesley_empty=0)
    run_steps(system, num_steps, p1, p2, plot_flag)
    return system

In [16]:
run_simulation(bikeshare, 60, 0.7, 0.2, False)

,value
olin,0
wellesley,12
olin_empty,16
wellesley_empty,0


## More for loops

`linspace` creates a NumPy array of equally spaced numbers.

In [193]:
p1_array = linspace(start=0, stop=1, num=5)
p1_array

array([ 0.  ,  0.25,  0.5 ,  0.75,  1.  ])

We can use an array in a `for` loop, like this:

In [194]:
for p1 in p1_array:
    print(p1)

0.0
0.25
0.5
0.75
1.0


This will come in handy in the next section.

**Exercise:** The function `linspace` is part of NumPy.  [You can read the documentation here](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linspace.html).

Use `linspace` to make an array of 10 equally spaced numbers from 1 to 10 (including both).

In [197]:
a_linear_space = linspace(1,10,10)
a_linear_space

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.])

**Exercise:** The `modsim` library provides a related function called `linrange`.  You can view the documentation by running the following cell:

In [199]:
help(linrange)

Help on function linrange in module modsim:

linrange(start=0, stop=None, step=1, **kwargs)
    Returns an array of evenly-spaced values in the interval [start, stop].
    
    This function works best if the space between start and stop
    is divisible by step; otherwise the results might be surprising.
    
    By default, the last value in the array is `stop` (at least approximately).
    If you provide the keyword argument `endpoint=False`, the last value
    in the array is `stop-step`. 
    
    start: first value
    stop: last value
    step: space between values
    
    Also accepts the same keyword arguments as np.linspace.  See
    https://docs.scipy.org/doc/numpy/reference/generated/numpy.linspace.html
    
    returns: array or Quantity



Use `linrange` to make an array of numbers from 1 to 10 with a step size of 2.

In [204]:
linrange(1,11,2,endpoint = False)

array([ 1.,  3.,  5.,  7.,  9.])

## Sweeping parameters

The following example runs simulations with a range of values for `p1`; after each simulation, it prints the number of unhappy customers at the Olin station:

In [434]:
p1_array = linspace(0, 1, 21)
p1_array

array([ 0.  ,  0.05,  0.1 ,  0.15,  0.2 ,  0.25,  0.3 ,  0.35,  0.4 ,
        0.45,  0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,
        0.9 ,  0.95,  1.  ])

In [435]:
for p1 in p1_array:
    system = run_simulation(p1=p1)
    print(p1, system.olin_empty)

0.0 0
0.05 0
0.1 0
0.15 0
0.2 0
0.25 0
0.3 0
0.35 0
0.4 2
0.45 9
0.5 8
0.55 14
0.6 12
0.65 18
0.7 16
0.75 22
0.8 24
0.85 28
0.9 34
0.95 38
1.0 37


Now we can do the same thing, but plotting the results instead of printing them.



In [437]:
newfig()
for p1 in p1_array:
    system = run_simulation(p1=p1)
    plot(p1, system.olin_empty, 'rs', label='olin')

As always, we should decorate the figure.  This version of `decorate` takes `xlabel` as a parameter, for reasons you will see soon.

In [16]:
def decorate(xlabel):
    legend(loc='best')
    label_axes(title='Olin-Wellesley Bikeshare',
               xlabel=xlabel, 
               ylabel='Number of unhappy customers')

In [439]:
decorate(xlabel='Arrival rate at Olin (p1 in customers/min)')

**Exercise:** Wrap this code in a function named `parameter_sweep` that takes an array called `p1_array` as a parameter.  It should create a new figure, run a simulation for each value of `p1` in `p1_array`, and plot the results.

Once you have the function working, modify it so it also plots the number of unhappy customers at Wellesley.  Looking at the plot, can you estimate a range of values for `p1` that minimizes the total number of unhappy customers?

In [17]:
def parameter_sweep(p1_array):
    #newfig() #commenting out newfig is a fun way to look at stochastic simulations.
    for p1 in p1_array:
        system = run_simulation(p1=p1)
        plot(p1, system.olin_empty, 'rs', label='Olin :(')
        plot(p1, system.wellesley_empty, 'bo', label='Wellesley :(')
    decorate(xlabel='Arrival rate at Olin (p1 in customers/min)')

In [481]:
parameter_sweep(p1_array)

In [62]:
# .25 is probably the ideal value for p1.

**Exercise:** Write a function called `parameter_sweep2` that runs simulations with `p1=0.2` and a range of values for `p2`.

Note: If you run `parameter_sweep2` a few times without calling `newfig`, you can plot multiple runs on the same axes, which will give you a sense of how much random variation there is from one run to the next. 

In [18]:
p2_array = linspace(0,1,21)
def parameter_sweep(p2_array):
    #newfig() 
    for p2 in p2_array: 
        system = run_simulation(p1=.25, p2=p2)
        plot(p2, system.olin_empty, 'rs', label='Olin :(')
        plot(p2, system.wellesley_empty, 'bo', label='Wellesley :(')
    decorate(xlabel='Arrival rate at Wellesley (p2 in customers/min)')

In [510]:
parameter_sweep(p2_array)

In [65]:
# .2 is probably the ideal value for p2, if p1 is .25.

**Exercise:** Hold `p1=0.4` and `p2=0.2`, and sweep a range of values for `num_steps`.

Hint: You will need a version of `run_simulation` that takes `num_steps` as a parameter.

Hint: Because `num_steps` is supposed to be an integer use `range` rather than `linrange`.

In [19]:
steps_array = range(0,120,10)
def parameter_sweep(steps_array):
    newfig() 
    for step_count in steps_array: 
        system = run_simulation(p1=.25, p2=.25, num_steps = step_count)
        plot(step_count, system.olin_empty, 'rs', label='Olin :(')
        plot(step_count, system.wellesley_empty, 'bo', label='Wellesley :(')
    decorate(xlabel='Number of elapsed minutes')
    print (system)

In [591]:
parameter_sweep(steps_array)

olin                9
wellesley           3
olin_empty          0
wellesley_empty    11
dtype: int64


In [379]:
# Solution goes here

**Exercise:** The code below runs a simulation with the same parameters 10 times and computes the average number of unhappy customers.

1.  Wrap this code in a function called `run_simulations` that takes `num_runs` as a parameter.

2.  Test `run_simulations`, and increase `num_runs` until the results are reasonably consistent from one run to the next.

3.  Generalize `run_simulations` so it also takes the initial value of `olin` as a parameter.

4.  Run the generalized version with `olin=12`.  How much do the two extra bikes decrease the average number of unhappy customers?

5.  Make a plot that shows the average number of unhappy customers as a function of the initial number of bikes at Olin.

OKAY SO.
When I defined run_simulation using "all 5 model parameters", I used the 5 parameters that go into run_steps, because p1 and p2 were both for run_steps so that seemed to be the thing to do. Then I did almost all of the following exercises and activities and this function worked just fine and there was no indication that I had written it wrong. Maybe this is just me being unable to distinguish between function and system variables, I really didn't know that I was supposed to use olin and wellesley as parameters. So someone please help me with that. I figured out my mistake when I asked a classmate for help and they suggested I look at the solutions (I copied that definition of run_simulation below so my solution would work.) Anyway, I would really like to know what piece of information I missed that led me to do a bunch of stuff wrong, because I hate having to just copy in something when I don't know why it was done that way.

In [20]:
def run_simulation(p1=0.4, p2=0.2, olin=10, wellesley=2, num_steps=60):
    bikeshare = System(olin=olin, wellesley=wellesley, 
                  olin_empty=0, wellesley_empty=0)
    run_steps(bikeshare, num_steps, p1, p2, plot_flag=False)
    return bikeshare

In [27]:
num_runs = 10
total = 0
for i in range(num_runs):
    system = run_simulation(p1=0.4, p2=0.2, olin=10, wellesley=2, num_steps=60)
    total += system.olin_empty + system.wellesley_empty
total / num_runs

4.0

In [21]:
def run_simulations(num_runs):
    total = 0
    for i in range(num_runs):
        system = run_simulation(p1=0.4, p2=0.2, olin=10, wellesley=2, num_steps=60)
        total += system.olin_empty + system.wellesley_empty
    return total / num_runs

In [73]:
average = run_simulations(1000)
print(average)

3.696


In [22]:
def run_simulations(num_runs, olin):
    total = 0
    for i in range(num_runs):
        system = run_simulation(p1=0.4, p2=0.2, olin=olin, wellesley=2, num_steps=60)
        total += system.olin_empty + system.wellesley_empty
    return total / num_runs

In [23]:
average = run_simulations(1000, 12)
print(average)

2.438


In [26]:
improvement =3.696-2.438
print(improvement)

1.258


In [27]:
bikes_array = range(20)
def parameter_sweep(bikes_array):
    newfig() 
    for bike_count in bikes_array: 
        average = run_simulations(1000, bike_count)
        plot(bike_count, average, 'rs', label='Struggle Factor')
    decorate(xlabel='Initial Bikes at Olin')

In [28]:
parameter_sweep(bikes_array)

IT WORKS BITCHESSSSSSSS

In [32]:
# Sorry, but you must understand,
notebook_num = 2
struggle_factor = notebook_num ** 100
print(struggle_factor)

1267650600228229401496703205376
